In [ ]:
import requests
from bs4 import BeautifulSoup
import sqlite3

conn = sqlite3.connect("kbo.db")
cur = conn.cursor()

def insert_pitcher_record(
    player_name, position, result, win, loss, save,
    inning, batters_faced, pitches, outs, hits_allowed,
    home_runs, walks, strikeouts, runs, earned_runs, era,
    game_id, team_id
):
    cur.execute("""
        INSERT INTO pitchers (
            player_name, position, result, win, loss, save,
            inning, batters_faced, pitches, outs, hits_allowed,
            home_runs, walks, strikeouts, runs, earned_runs, era,
            game_id, team_id
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        player_name, position, result, win, loss, save,
        inning, batters_faced, pitches, outs, hits_allowed,
        home_runs, walks, strikeouts, runs, earned_runs, era,
        game_id, team_id
    ))

    conn.commit()


def scrape_pitchers(game_id, team_id, url):
    headers = {"User-Agent": "Mozilla/5.0"}
    res = requests.get(url, headers=headers)
    soup = BeautifulSoup(res.text, "html.parser")

    # "투수 기록"이라는 제목 아래의 테이블 선택 (상황에 따라 조정 필요)
    pitcher_tables = soup.select("table.tData")  # 보통 GameCenter의 테이블 클래스

    for table in pitcher_tables:
        rows = table.select("tbody tr")

        for row in rows:
            cols = [td.get_text(strip=True) for td in row.select("td")]

            # TOTAL 행 건너뛰기
            if cols[0] == "TOTAL":
                continue

            player_name = cols[0]
            position = cols[1]
            result = cols[2]
            win = int(cols[3])
            loss = int(cols[4])
            save = int(cols[5])
            inning = cols[6]
            batters_faced = int(cols[7])
            pitches = int(cols[8])
            outs = int(cols[9])
            hits_allowed = int(cols[10])
            home_runs = int(cols[11])
            walks = int(cols[12])
            strikeouts = int(cols[13])
            runs = int(cols[14])
            earned_runs = int(cols[15])
            era = 0.0 if cols[16] == "-" else float(cols[16])

            insert_pitcher_record(
                player_name, position, result, win, loss, save,
                inning, batters_faced, pitches, outs, hits_allowed,
                home_runs, walks, strikeouts, runs, earned_runs, era,
                game_id, team_id
            )

    print(f"✔ game_id={game_id}, team_id={team_id} 투수 기록 저장 완료!")


ModuleNotFoundError: No module named 'pandas'

In [5]:
import requests
import json

url = "https://www.koreabaseball.com/GameCenter/Review1"

payload = {
    "gameId": "20241006NCSSG",
    "seriesCode": "",
    "detailType": "2"
}

headers = {
    "Content-Type": "application/json; charset=UTF-8",
    "User-Agent": "Mozilla/5.0"
}

res = requests.post(url, headers=headers, data=json.dumps(payload))
print(res.text)


ï»¿<!--[if IE 7]><html class="ie7" lang="ko"><![endif]-->
<!--[if IE 8]><html class="ie8" lang="ko"><![endif]-->
<!--[if IE 9]><html class="ie9" lang="ko"><![endif]-->
<!DOCTYPE html>
<html lang="ko">
<head>
    <meta charset="UTF-8">
    <meta http-equiv="X-UA-Compatible" content="IE=Edge" />
    <title>ìë¬ | KBOííì´ì§ </title>
    <style type="text/css">
        body {
            background: #fff;
        }

        body {
            font-size: 12px;
            font-family: 'Nanum Gothic',sans-serif;
            color: #333;
        }

        a {
            color: #333;
            text-decoration: none;
            display: inline-block;
        }

            a:active, a:hover {
                text-decoration: none;
            }

        body {
            margin: 0;
            padding: 0;
        }

        .errorbox {
            width: 100%;
            margin: 0 auto;
        }
        /* width:1013px;*/
        .elogo {
            text-align: center;
        

해결 필요함..

In [1]:
import pandas as pd
import sqlite3

# ==== 1. DB 연결 ====
conn = sqlite3.connect("kbo.db")
cur = conn.cursor()

# ==== 2. 엑셀 불러오기 ====
df = pd.read_excel("pitchers_data.xlsx")   # 시트명이 있다면 sheet_name="Sheet1"

# ==== 3. INSERT 함수 ====
def insert_pitcher(row):
    cur.execute("""
        INSERT INTO pitchers (
            player_id, player_name, at_bats, hits, rbi, run, avg,
            game_id, team_id
        ) VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, (
        row["player_id"],
        row["player_name"],
        row["at_bats"],
        row["hits"],
        row["rbi"],
        row["run"],
        row["avg"],
        row["game_id"],
        row["team_id"]
    ))

# ==== 4. 전체 행 반복 ====
for idx, row in df.iterrows():
    insert_pitcher(row)

conn.commit()
conn.close()

print("✔ 엑셀 데이터를 pitchers 테이블에 성공적으로 삽입했습니다.")


KeyError: 'player_id'